#gureBERT

In [4]:
# gureBERT

!git clone --recursive  https://github.com/zmwebdev/gureBERT
%cd gureBERT

!pip install sentencepiece
!install -d spModels
# eu
!python src/sentence-split.py --config eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config eu.config.ini

!head -n 100 spModels/eu.vocab

# en-eu
!python src/sentence-split.py --config en-eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config en-eu.config.ini

!head -n 100 spModels/en-eu.vocab


Cloning into 'gureBERT'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 225 (delta 59), reused 85 (delta 29), pack-reused 109
Receiving objects: 100% (225/225), 7.02 MiB | 18.01 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Submodule 'bert' (https://github.com/google-research/bert.git) registered for path 'bert'
Cloning into '/content/gureBERT/bert'...
remote: Enumerating objects: 329, done.        
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 275.96 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (181/181), done.
Submodule path 'bert': checked out '0fce551b55caabcfba52c61e18f34b541aef186a'
/content/gureBERT


In [0]:
from google.colab import auth
auth.authenticate_user()

W0713 05:13:49.337402 140654018004864 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# google bucket
!gsutil cp -r gs://gurebert/gureBERT/spModels .

Copying gs://gurebert/gureBERT/spModels/en-eu.model...
Copying gs://gurebert/gureBERT/spModels/en-eu.vocab...
Copying gs://gurebert/gureBERT/spModels/eu.model...
Copying gs://gurebert/gureBERT/spModels/eu.vocab...
\ [4 files][  2.9 MiB/  2.9 MiB]                                                
Operation completed over 4 objects/2.9 MiB.                                      


In [0]:
import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

tokenizer = tokenization.FullTokenizer(
    model_file="spModels/eu.model",
    vocab_file="spModels/eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da. Nere baratzak 3 km² ditu. ni@ni.eus"

print(tokenizer.tokenize(text1))

Loaded a trained SentencePiece model.
['▁nere', '▁ko', 'txea', '▁ait', 'onaren', '▁etxe', '▁alboan', '▁dago', ',', '▁bere', '▁kolorea', '▁gorria', '▁da', '.', '▁nere', '▁bar', 'atzak', '▁3', '▁km', '²', '▁ditu', '.', '▁ni', '@', 'ni', '.', 'eus']


In [0]:
tokenizer_en_eu = tokenization.FullTokenizer(
    model_file="spModels/en-eu.model",
    vocab_file="spModels/en-eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da."

print(tokenizer_en_eu.tokenize(text1))

Loaded a trained SentencePiece model.
['▁ne', 're', '▁ko', 'txea', '▁a', 'it', 'on', 'aren', '▁etxe', '▁', 'albo', 'an', '▁dago', ',', '▁bere', '▁kolore', 'a', '▁gorria', '▁da', '.']


In [0]:
text1 = "The Italian cities of Milan and Cortina d'Ampezzo are chosen as the joint hosts of the 2026 Winter Olympics and Winter Paralympics."

print("EN-EU: {}".format(tokenizer_en_eu.tokenize(text1)))
print("EU: {}".format(tokenizer.tokenize(text1)))

print("EN-EU: {}".format(len(tokenizer_en_eu.tokenize(text1))))
print("EU: {}".format(len(tokenizer.tokenize(text1))))

EN-EU: ['▁the', '▁italian', '▁cities', '▁of', '▁milan', '▁and', '▁cort', 'ina', '▁d', "'", 'amp', 'e', 'zzo', '▁are', '▁chosen', '▁as', '▁the', '▁joint', '▁hosts', '▁of', '▁the', '▁20', '26', '▁winter', '▁olympics', '▁and', '▁winter', '▁para', 'lympic', 's', '.']
EU: ['▁the', '▁italian', '▁cit', 'ies', '▁of', '▁milan', '▁and', '▁cor', 'tina', '▁d', "'", 'amp', 'ez', 'zo', '▁are', '▁ch', 'osen', '▁as', '▁the', '▁jo', 'int', '▁hos', 'ts', '▁of', '▁the', '▁20', '26', '▁winter', '▁olym', 'p', 'ics', '▁and', '▁winter', '▁paral', 'ym', 'p', 'ics', '.']
EN-EU: 31
EU: 38


In [5]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

W0713 17:24:55.675337 140526703961984 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU address is grpc://10.51.131.226:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6810479753173132532),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9581148650088822417),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14433261477188369500),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10471305944331051035),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 290337485020010624),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8844700230351949478),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 17633311837389007867),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 501559178114276546),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 74300561114596

In [0]:
GS = 'gs://gurebert/gureBERT'
  
#!gsutil cp -r spModels $GS/
#!gsutil cp -r corpus $GS/

## EU

In [0]:
!python3 src/create_pretraining_data.py \
    --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
    --output_file=$GS/pretraining.tf.data \
    --model_file=spModels/eu.model \
    --vocab_file=spModels/eu.vocab \
    --do_lower_case=True

In [0]:
#!install -d gureBERT

!python src/run_pretraining.py \
  --config_file eu.config.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT/test \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=256 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=100 \
  --num_warmup_steps=10 \
  --save_checkpoints_steps=100 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
#  --num_train_steps=1000000 \

# num_train_steps ? zenbatekoa?
# 

In [0]:
#pretraining jarraipena 

GS = 'gs://gurebert/gureBERT'

!python src/run_pretraining.py \
  --config_file eu.config.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT/test \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=200 \
  --num_warmup_steps=10 \
  --save_checkpoints_steps=200 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/eu.gureBERT/test \
#  --num_train_steps=1000000 \


## EN-EU

In [0]:
FILES = "./corpus/en-eu/2014wiki.eu.sent_splited,./corpus/en-eu/2019wiki-10k.en.sent_splited"

!python3 src/create_pretraining_data.py \
    --input_file={FILES} \
    --output_file=$GS/pretraining-en_eu.tf.data \
    --model_file=spModels/en-eu.model \
    --vocab_file=spModels/en-eu.vocab \
    --do_lower_case=True

In [0]:
!python src/run_pretraining.py \
  --config_file en-eu.config.ini \
  --input_file=$GS/pretraining-en_eu.tf.data \
  --output_dir=$GS/en-eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/en-eu.gureBERT/ \
  #--num_train_steps=1000000 \


## SQuAD

In [0]:
# pre training SQuAD

# https://github.com/google-research/bert#squad-20

#!git clone https://github.com/zmwebdev/bert.git
#%cd bert

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
# evaluation script: download file from a url that returns a save dialog box : https://superuser.com/questions/795265/download-file-from-a-url-that-returns-a-save-dialog-box#795269
!wget --content-disposition https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/


In [0]:
# sentenpiece tokenizer erabili behar da!!: src/run_squad.py

!gsutil cp -r gs://gurebert/gureBERT/spModels .

 
!python src/run_squad.py \
  --vocab_file=./spModels/en-eu.vocab \
  --model_file=./spModels/en-eu.model \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  --init_checkpoint=gs://gurebert/gureBERT/squad/ \
    

## wordpiece erabiliz

- https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
- https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt




In [0]:
# eu
#!python src/sentence-split.py --config eu.config.ini --do_lower_case 
!python src/train-sentencepiece-wordpiece.py --config eu.config.ini

!head -n 100 spModels/eu.vocab

# en-eu
!python src/sentence-split.py --config en-eu.config.ini --do_lower_case 
!python src/train-sentencepiece-wordpiece.py --config en-eu.config.ini

!head -n 100 spModels/en-eu.vocab

### EU

In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/en-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

In [0]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [0]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

VOC_FNAME = "vocab.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")


In [0]:
#import string

#bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

##########################
#
#   Falta diren sinbolo gehiketa modu automatikoan egin daiteke 
#   sentenpiece parametroak erabiliz:
#   https://github.com/google/sentencepiece/blob/d4dd947fe71c4fa4ee24ad8297beee32887d8828/python/sentencepiece_python_module_example.ipynb
#   https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
#
##########################

#ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
## puntuazio sinboloak gehitu 
#ctrl_symbols_end = list(string.punctuation)
##ctrl_symbols = ["[UNK]"]
#bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab = ctrl_symbols + bert_vocab

## https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
## Additionally, we append some placeholder tokens to the vocabulary. 
## Those are useful if one wishes to update the pre-trained model with new, 
## task-specific tokens. In that case, the placeholder tokens are replaced with new real ones, 
## the pre-training data is re-generated, and the model is fine-tuned on new data.
#
# Nola egiten da re-generated?
#
#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]

#print(len(bert_vocab))

#VOC_FNAME = "vocab.txt"
#with open(VOC_FNAME, "w") as fo:
#  for token in bert_vocab:
#    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
#text1 = "Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili hitzhauezdaexixtitzen !~gg _*] "
#text1 = "44.579.000 km²ko eremua eta 4.140.336.501 biztanle ditu"
#text1 = "biologia izena grezierazko bi osagai erabiliz eratua da: βίος, bios, «bizitza»; eta -λογία, -logia, «azterketa»."
#text1 = "手洗い"
text1 = "ἀριστοτέλης meta+fisika km²"
print("num:{}, tokenak:{}".format(len(bert_tokenizer.tokenize(text1)),bert_tokenizer.tokenize(text1)))
print("num:{}, tokenak:{}".format(len(tokenizer.tokenize(text1)),tokenizer.tokenize(text1)))

# [UNK] ak agertzen dira wordpiece-ren kasuan token hori vocab-en ez badago, adibidez '~' karakterea. 
# Sentencepiece-ren kasuan, nahiz eta bere vocab-en ez egon ($ !cat spModels/en-eu.vocab | grep '~') tokenizatu egiten du
# hau da, sentencepuecek 'dena' tokenizatzen du nahiz eta bere vocab-ean ez egon
#
# baina 'km²' corpusean agertzen da eta wordpiecek ez du tokenizatzen [UNK] bezala agertzen da. berdin 'ἄστρον' (ikusi beherago)
# 'km²' ez da sentencepieceko en-eu.vocab-ean agertzen, baizik eta 'km' eta '2'. hau da: 'km²' -> ['km'] ['2'] tokenizatzen du eta BERT-ek ez
# beraz kasu hauetan (karaktere arraroak? ...) 'eskuz' sartu beharko dira wordpiecen?
# BERT base originaleko vocab.txt (edo germanBERT) aztertu eta ikusi bertan token hori dagoen...

In [0]:
# https://github.com/google-research/bert/blob/master/create_pretraining_data.py
# https://github.com/google-research/bert/blob/master/create_pretraining_data.py#L179

import tensorflow as tf
import sys
sys.path.append('bert')

import create_pretraining_data
import tokenization

bert_tokenizer = tokenization.FullTokenizer(vocab_file=VOC_FNAME, do_lower_case=True)

#all_documents = [[]]
all_documents = []
input_files = ['./corpus/eu/2014wiki.eu.sent_splited']
for input_file in input_files:
  with tf.gfile.GFile(input_file, "r") as reader:
    while True:
      line = tokenization.convert_to_unicode(reader.readline())
      if not line:
        break
      line = line.strip()

      # Empty lines are used as document delimiters
      #if not line:
      #  all_documents.append([])
      tokens = bert_tokenizer.tokenize(line)
      if tokens:
        #all_documents[-1].append(tokens)
        all_documents += tokens
        #print(tokens)

# Remove empty documents
all_documents = [x for x in all_documents if x]
#rng = random.Random(1234)
#rng.shuffle(all_documents)

print("token kop: {}".format(len(all_documents)))
print("[UNK] kop: {}".format(all_documents.count('[UNK]')))

In [0]:
# 



def bert_tokenize_corpus(filepath):
  tokens = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      token_line = bert_tokenizer.tokenize(line)
      tokens += token_line
      if '[UNK]' in token_line:
        print("testua: {}".format(line))
        print("tokenak: {}".format(token_line))
        print("---------------------------------------------------------")
        
  return tokens

def sentencepiece_tokenize_corpus(filepath):
  tokens = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      tokens += tokenizer.tokenize(line)      
  return tokens


bert_tokens = bert_tokenize_corpus("./corpus/eu/2014wiki.eu.sent_splited")
sentencepiece_tokens = sentencepiece_tokenize_corpus("./corpus/eu/2014wiki.eu.sent_splited")

print("wordpiece tokens: {}".format(len(bert_tokens)))
print("sentencepiece tokens: {}".format(len(sentencepiece_tokens)))

print("Wordpiece [UNK] kop: {}".format(bert_tokens.count('[UNK]')))

In [0]:
!python bert/create_pretraining_data.py \
  --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
  --output_file=$GS/wordpiece/pretraining.tf.data \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining.tf.data \ \
  --output_dir=$GS/wordpiece/model \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  #--num_train_steps=1000000 \
  #--init_checkpoint=$GS/wordpiece/model \

### EN-EU

In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/en-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

In [0]:
# hau errepasatu. adibidez @ ez dut ondo egiten... [UNK] agertzen da...
# adibidez, 

#import string
#def parse_sentencepiece_token(token):
#    if token.startswith("▁"):
#        return token[1:]
#    else:
#        if token in string.punctuation:
#            return token
#        else:
#            return "##" + token
          

def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [0]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
# puntuazio sinboloak gehitu 
ctrl_symbols_end = list(string.punctuation)
#ctrl_symbols = ["[UNK]"]
bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

In [0]:
VOC_FNAME = "vocab-en_eu.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
print(bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili "))

In [0]:
FILES = "./corpus/en-eu/2014wiki.eu.sent_splited,./corpus/en-eu/2019wiki-10k.en.sent_splited"

!python bert/create_pretraining_data.py \
  --input_file={FILES} \
  --output_file=$GS/wordpiece/pretraining-en_eu.tf.data \
  --vocab_file=vocab-en_eu.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining-en_eu.tf.data \ \
  --output_dir=$GS/wordpiece/model-en_eu \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/wordpiece/model-en_eu/ \
  #--num_train_steps=1000000 \
  

### SQuAD

In [0]:
# data English da!! baino kodea badabil 

!python bert/run_squad.py \
  --vocab_file=vocab-en_eu.txt \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/wordpiece/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  --init_checkpoint=gs://gurebert/gureBERT/wordpiece/model-en_eu/ \

# gureBERT EN-EU Sorkuntza

Ideia honakoa da: colab erabiliz BERT EN-EU entrenatzea. Ikusi:

- gureBERT japanese begiratu wikipediatik corpusaren prozesamendua
- gureBERT wordpiece nola egin begiratu
- https://github.com/zmwebdev/chatbot/issues/22#issuecomment-507390915

## Euskarazko vocab.txt sortu

Euskarazko vocab.txt sortu euskarazko wikiedia osoa hartuta

In [0]:
#

!cp eu.config.ini config.ini

!mkdir data
!python src/data-download-and-extract.py

In [0]:
#!bash src/file-preprocessing.sh
!python src/sentence-split.py --config eu.config.ini --do_lower_case 

####
!rm -Rf corpus/eu
!gsutil -m cp -r gs://gurebert/gureBERT/corpus/eu/[A-Z][A-Z] corpus/eu
####

!pip install sentencepiece
#!python src/train-sentencepiece-wordpiece.py --config eu.config.ini
!python src/train-sentencepiece-wordpiece.py --config eu.config.ini --tokenized True

In [0]:
# gs-ra kopiatu

!gsutil cp -r spModels/* gs://gurebert/gureBERT/spModels
!gsutil cp -r corpus/eu/* gs://gurebert/gureBERT/
  
# gs-tik kopiatu
!gsutil cp -r gs://gurebert/gureBERT/spModels .
!gsutil -m cp -r gs://gurebert/gureBERT/corpus/eu/[A-Z][A-Z] corpus/eu  # begiratu ea corpus/eu -n gordetzen dituen
#!gsutil cp -r gs://gurebert/gureBERT/corpus/eu corpus/eu


Copying file://spModels/eu.model [Content-Type=application/octet-stream]...
Copying file://spModels/eu.vocab [Content-Type=application/octet-stream]...
-
Operation completed over 2 objects/938.9 KiB.                                    


In [0]:
# sentencepiece2wordpiece

import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))


#
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token
      


##########################
#
#   Falta diren sinbolo gehiketa modu automatikoan egin daiteke 
#   sentenpiece parametroak erabiliz:
#   https://github.com/google/sentencepiece/blob/d4dd947fe71c4fa4ee24ad8297beee32887d8828/python/sentencepiece_python_module_example.ipynb
#   https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
#
##########################


gure_bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

#
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]  # hau automatikoki egin daiteke begiratu goiko url-a
#ctrl_symbols = ["[PAD]","[CLS]","[SEP]","[MASK]"]  # hau automatikoki egin daiteke begiratu goiko url-a

import string
punctuation_list = list(string.punctuation)

bert_vocab = ctrl_symbols + gure_bert_vocab + punctuation_list

unused_tokens = ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]

bert_vocab = ctrl_symbols + unused_tokens + gure_bert_vocab + punctuation_list

'''
################################
# BERT base uncaset vocab-etik (model-bert-base-uncased/vocab.bert.base.txt)
# lehenengo tokenak 'the' tokenetik hasierara
x = !cat model-bert-base-uncased/vocab.bert.base.txt| grep -n '^the$' | cut -d: -f1
n = int(x[0])-1
bert_tokens = !cat model-bert-base-uncased/vocab.bert.base.txt| head -n {n} 
# azkenengo tokenak 'necessitated' tik bukaera
x = !cat model-bert-base-uncased/vocab.bert.base.txt| grep -n '^necessitated$' | cut -d: -f1
n = int(x[0])+1
nn = !cat model-bert-base-uncased/vocab.bert.base.txt| wc -l
n = int(nn[0])-n+1
bert_tokens_azken = !cat model-bert-base-uncased/vocab.bert.base.txt| tail -n {n} 
  
# remove [unusedX]
bert_tokens = [ elem for elem in bert_tokens if not elem.startswith( '[unused' )]
bert_control_tokens = bert_tokens[0:5]
bert_tokens = bert_tokens[5:-1]

#print("BERT control tokens: {}".format(bert_control_tokens))
#print("BERT tokens: {}".format(bert_tokens))
#

# ez daudenak aukeratu
bert_tokens = [ elem for elem in bert_tokens if not elem in gure_bert_vocab]
bert_tokens_azken = [ elem for elem in bert_tokens_azken if not elem in gure_bert_vocab]


# UNUSED_X ak sortzen dira etorkizunean token berriak gehitu behar badira
bert_vocab_tmp = bert_control_tokens + bert_tokens + gure_bert_vocab + bert_tokens_azken
unused_tokens += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab_tmp))]

# dena batu
bert_vocab = bert_control_tokens + unused_tokens + bert_tokens + gure_bert_vocab + bert_tokens_azken
#################################
'''

VOC_FNAME = "wpModels/vocab.eu.txt"
with open(VOC_FNAME, "w", encoding='utf-8') as fo:
  for token in bert_vocab:
    fo.write(token+"\n")
    
#
#  SORTZEN DEN FITXATEGIAN txt KARAKTERA 'ARRAROAK'(beriratu amaierakoak) AGERTZEN DIRA BERT base-rekin KONPARATU!! 
#

#
#!mv vocab.eu.txt wpModels
!gsutil cp -r wpModels/vocab.eu.txt gs://gurebert/gureBERT/wpModels
  
#!gsutil cp -r gs://gurebert/gureBERT/wpModels/vocab.eu.txt .

#!head -n 100 wpModels/vocab.eu.txt
#!tail -n 100 wpModels/vocab.eu.txt

#!cat wpModels/vocab.eu.txt | grep -n \\##²

print("Sample tokens: {}".format(random.sample(bert_vocab, 10)))


Learnt vocab size: 29999
Sample tokens: ['モ', '▁doktoreak', '▁diputatu', '▁akusazio', '▁olerki', '▁1876', 'gau', '▁bakoitzeko', '▁ramirez', 'र']
Copying file://wpModels/vocab.eu.txt [Content-Type=text/plain]...
/ [1 files][246.4 KiB/246.4 KiB]                                                
Operation completed over 1 objects/246.4 KiB.                                    
Sample tokens: ['##โ', 'esleitze', '##capelle', 'gehiagoko', 'kashiwa', '##隊', 'mugak', 'eraztun', 'molde', 'kantauriar']


In [0]:
from bert import tokenization

VOC_FNAME = "wpModels/vocab.eu.txt"

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
text1 = "Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili hitzhauezdaexixtitzen !~gg _*] "
#text1 = "44.579.000 km²ko eremua eta 4.140.336.501 biztanle ditu"
#text1 = "k m²-ko km²-ko bainnn "
#text1 = "biologia izena grezierazko bi osagai erabiliz eratua da: βίος, bios, «bizitza»; eta -λογία, -logia, «azterketa»."
#text1 = "手洗い"
#text1 = "kaña bat nahi dut"
#text1 = "ἀριστοτέλης meta+fisika km²"
print("num:{}, tokenak:{}".format(len(bert_tokenizer.tokenize(text1)),bert_tokenizer.tokenize(text1)))


# [UNK] ak agertzen dira wordpiece-ren kasuan token hori vocab-en ez badago, adibidez '~' karakterea. 
# Sentencepiece-ren kasuan, nahiz eta bere vocab-en ez egon ($ !cat spModels/en-eu.vocab | grep '~') tokenizatu egiten du
# hau da, sentencepuecek 'dena' tokenizatzen du nahiz eta bere vocab-ean ez egon
#
# baina 'km²' corpusean agertzen da eta wordpiecek ez du tokenizatzen [UNK] bezala agertzen da. berdin 'ἄστρον' (ikusi beherago)
# 'km²' ez da sentencepieceko en-eu.vocab-ean agertzen, baizik eta 'km' eta '2'. hau da: 'km²' -> ['km'] ['2'] tokenizatzen du eta BERT-ek ez
# beraz kasu hauetan (karaktere arraroak? ...) 'eskuz' sartu beharko dira wordpiecen? ikusi hurrengo 'code cell'...
# (BERT base originaleko vocab.txt (edo germanBERT) aztertu eta ikusi bertan token hori dagoen...)

num:39, tokenak:['nere', 'kotxe', '##a', 'aitona', '##ren', 'etxe', 'albo', '##an', 'dago', '!', ',', 'eta', 'bere', 'kolorea', 'gorria', 'da', '.', 'ni', '@', 'ni', '.', 'eu', '##s', 'erabili', 'hitz', '##hau', '##ez', '##dae', '##xi', '##x', '##ti', '##tzen', '!', '~', 'g', '##g', '_', '*', ']']


In [0]:
# corpus guztian [UNK] -ak aurkitu eta ikusi zein hitz diren. Hauek [UNUSED] horietak jarri (edo amaieran gehitu eta gero jarri [UNUSED]-ak)
# hurrengo 

def find_unk_sentences(filepath):
  unk_lines = []
  tokens = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      token_line = bert_tokenizer.tokenize(line)
      tokens += token_line
      if '[UNK]' in token_line:
        print("testua: {}".format(line))
        print("tokenak: {}".format(token_line))
        print("---------------------------------------------------------")
        unk_lines.append(line)
  return unk_lines


def find_unk_word(unk_lines):
  unk_words = []
  for line in unk_lines:
    for w in line:
      token = bert_tokenizer.tokenize(w)
      if '[UNK]' in token:
        unk_words.append(w)
  return set(unk_lines)


import glob

def _get_text_file(text_dir=''):
    file_list = glob.glob(f'{text_dir}/*/*.sent_splited')
    files = ",".join(file_list)
    return files

files = _get_text_file('./corpus/eu/')
#### HAU KENDU!! proba bat da!!
#files = 'corpus/eu/AA/wiki_00.sent_splited'


#unk_words = []
unk_sentences = []
for fs in files.split(","):
  #tmp = find_unk_sentences(fs)
  #tmp2 = find_unk_word(tmp)
  #unk_words = unk_words.append(find_unk_word(find_unk_sentences(fs)))
  unk_sentences.append(find_unk_sentences(fs))

print("Wordpiece [UNK] sentences len: {}".format(len(unk_sentences)))
#print("Wordpiece [UNK] words: {}".format(unk_words))
 

#hitza zein den aurkite eta zerrenda bat egin. gero vocab-ean sartu
#print("wordpiece tokens: {}".format(len(bert_tokens)))
#print("sentencepiece tokens: {}".format(len(sentencepiece_tokens)))

print(tmp2)

## pre-train



In [0]:
#Creating data for pretraining
#Create .tfrecord files for pretraining. For longer sentence data, replace the value of max_seq_length with 512.

#!cat src/creating_data_for_pretraining.sh
#!bash src/creating_data_for_pretraining.sh BERT-ena erabili!!!!


# The max_predictions_per_seq is the maximum number of masked LM predictions per sequence. 
# You should set this to around max_seq_length * masked_lm_prob 
# (the script doesn't do that automatically because the exact value needs to be passed to both scripts).


MAX_SEQ_LEN = 128
#MAX_SEQ_LEN = 512
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS_PER_SEQ = round(MAX_SEQ_LEN * MASKED_LM_PROB)


#input_files_list = glob.glob(f'corpus/eu/*/*.sent_splited')
#input_files_list = glob.glob(f'gs://gurebert/gureBERT/corpus/*/*.sent_splited')
input_files_list = !gsutil ls gs://gurebert/gureBERT/corpus/eu/*/*.sent_splited
#input_files_list = ['gs://gurebert/gureBERT/corpus/AA/wiki_00.sent_splited']

for file in input_files_list:
  out_file = file + '.tfrecord'
  !python bert/create_pretraining_data.py \
    --input_file={file} \
    --output_file={out_file} \
    --vocab_file=wpModels/vocab.eu.txt \
    --do_lower_case=True \
    --max_seq_length={MAX_SEQ_LEN} \
    --max_predictions_per_seq={MAX_PREDICTIONS_PER_SEQ} \
    --masked_lm_prob={MASKED_LM_PROB} \
    --random_seed=12345 \
    --dupe_factor=5



In [7]:
# continue creating data for pretraining

x = !gsutil ls gs://gurebert/gureBERT/corpus/eu/*/*.tfrecord | wc -l
y = !gsutil ls gs://gurebert/gureBERT/corpus/eu/*/*.sent_splited | wc -l

if (x == y):
  print('All ptretraining done!')
else:
  # TODO
  none



All ptretraining done!


In [10]:
OUTPUT_GCS = 'gs://gurebert/gureBERT/wordpiece/model'

MAX_SEQ_LEN = 128
#MAX_SEQ_LEN = 512
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS_PER_SEQ = round(MAX_SEQ_LEN * MASKED_LM_PROB)

files = !gsutil ls gs://gurebert/gureBERT/corpus/eu/*/*.tfrecord
files = ",".join(files)

# OR copy locally
#!gsutil -m cp -r gs://gurebert/gureBERT/corpus/eu/*/*.tfrecord /tmp
#files = glob.glob(f'/tmp/*.tfrecord')
#files = ",".join(files)

##files = '/tmp/wiki_35.sent_splited.tfrecord'

bert_config = {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 32000
}


import json
with open('bert_config.json', 'w', encoding='utf-8') as f:
    json.dump(bert_config, f, ensure_ascii=False, indent=4)

!python bert/run_pretraining.py \
  --input_file={files} \
  --output_dir={OUTPUT_GCS} \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq={MAX_PREDICTIONS_PER_SEQ} \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  #--num_train_steps=1000000 \
  #--init_checkpoint={OUTPUT_GCS} \


W0713 17:26:42.971759 140353320949632 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0713 17:26:42.972923 140353320949632 deprecation_wrapper.py:119] From bert/run_pretraining.py:493: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0713 17:26:42.973571 140353320949632 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0713 17:26:42.973752 140353320949632 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0713 17:26:42.973908 140353320949632 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0713 17:26:42.974625 14035332094963

In [14]:
# jarraitu pre-training

!python bert/run_pretraining.py \
  --input_file={files} \
  --output_dir={OUTPUT_GCS} \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq={MAX_PREDICTIONS_PER_SEQ} \
  --num_train_steps=250000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint={OUTPUT_GCS} \
  #--num_train_steps=1000000 \
  

W0713 22:24:14.442707 140417550112640 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0713 22:24:14.443955 140417550112640 deprecation_wrapper.py:119] From bert/run_pretraining.py:493: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0713 22:24:14.444674 140417550112640 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0713 22:24:14.444829 140417550112640 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0713 22:24:14.444988 140417550112640 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0713 22:24:14.445811 14041755011264

# gureBERT EN-EU Sorkuntza

Irakurri:
- https://github.com/google-research/bert/blob/master/multilingual.md#data-source-and-sampling:

> To balance these two factors, we performed exponentially smoothed weighting of the data during pre-training data creation (and WordPiece vocab creation). In other words, let's say that the probability of a language is P(L), e.g., P(English) = 0.21 means that after concatenating all of the Wikipedias together, 21% of our data is English. We exponentiate each probability by some factor S and then re-normalize, and sample from that distribution. In our case we use S=0.7. So, high-resource languages like English will be under-sampled, and low-resource languages like Icelandic will be over-sampled. E.g., in the original distribution English would be sampled 1000x more than Icelandic, but after smoothing it's only sampled 100x more.

- https://github.com/google-research/bert/blob/master/multilingual.md#tokenization:

> For tokenization, we use a 110k shared WordPiece vocabulary. The word counts are weighted the same way as the data, so low-resource languages are upweighted by some factor. We intentionally do not use any marker to denote the input language (so that zero-shot training can work).


# gureBERT (japanese) erabiliz

In [0]:
#################################################################
# gureBERT japanese

!cd /content/

!git clone --recursive https://github.com/zmwebdev/bert-japanese
%cd bert-japanese



In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!python3 src/data-download-and-extract.py
!bash src/file-preprocessing.sh

#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 data/wiki/AA/wiki_00

# Oharra: esaldiak lerro bakarrean jarri behar dira
#!cat data/wiki/AA/wiki_00

In [0]:
# wiki-eu.vocab eta wiki-eu.model sortzen ditu

!pip install sentencepiece
!python3 src/train-sentencepiece.py

In [0]:
!gsutil cp -r data/ gs://gurebert/gureBERT/
!gsutil cp -r model/ gs://gurebert/gureBERT/

In [0]:
!mkdir model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.vocab model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.model model
!head -n 50 model/wiki-eu.vocab

In [0]:
#https://github.com/zmwebdev/bert-japanese/blob/master/notebook/check-trained-tokenizer.ipynb
!pip install sentencepiece

import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

text1 = "Nere kotxea aitonaren etxe alboan dago"
text2 = "Gorria da gure etxearen inguruan dagoen lorearen kolorea"

tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=True)

tokenizer.tokenize(text1)

In [0]:
tokenizer.tokenize(text2)

In [0]:
#do_lower_case=False jarrita tokenizazioa okerragoa da
tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=False)

tokenizer.tokenize(text1)

In [0]:
tokenizer.tokenize(text2)

In [0]:
# convert to WordPiece (kodean errorea dago). begiratu https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

!git clone https://github.com/kwonmha/bert-vocab-builder.git

!python bert-vocab-builder/subword_builder.py \
--corpus_filepattern model/wiki-eu.vocab \
--output_filename model/wiki-eu-wordpiece.vocab
#--min_count {minimum_subtoken_counts}

# https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

In [0]:
#Creating data for pretraining
#Create .tfrecord files for pretraining. For longer sentence data, replace the value of max_seq_length with 512.
!cat creating_data_for_pretraining.sh
!bash creating_data_for_pretraining.sh

In [0]:
# copy to gs
!gsutil cp -r data gs://gurebert/gureBERT
!gsutil cp -r model gs://gurebert/gureBERT
  

In [0]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  


In [0]:
INPUT_DATA_GCS = 'gs://gurebert/gureBERT/data/wiki'
TARGET_DIRS = [
  'AA',
  'AB',
  'AC'
]

MAX_SEQ_LEN = 128
#MAX_SEQ_LEN = 512


INPUT_FILE = ','.join( [ '{}/{}/all-maxseq{}.tfrecord'.format(INPUT_DATA_GCS, elem, MAX_SEQ_LEN) for elem in TARGET_DIRS] )

OUTPUT_GCS = 'gs://gurebert/gureBERT/model'

In [0]:
# Adding whole word masking aldaketa egin dute BERT-en. update egin behar da!!!
# Begiratu https://github.com/google-research/bert/commit/0fce551b55caabcfba52c61e18f34b541aef186a

!python src/run_pretraining.py \
  --input_file={INPUT_FILE} \
  --output_dir={OUTPUT_GCS} \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --num_tpu_cores=8 \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq=20 \
  --num_train_steps=1400000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4

In [0]:
##########################################
##########################################

# BERT-ekin konpatiblea den vocab sortzeko prozedura. wordpiece _ -> ##. 
# Honen abantaila BERT kodearen aldaketak eta run_squad.py, ... ezer aldatu gabe ibiliko dela da.
# berez ez da wordpiece, sentencepiece baizik baina sintaktikoki konpatiblea
# ikusi: https://colab.research.google.com/drive/1-uLyGTnz2K4gx3su6Qc00in9QCa4b5Kh#scrollTo=9S4CiOh3RzFW


In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('model/wiki-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

In [0]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

In [0]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

In [0]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago")



In [0]:
#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 wiki_00
# ez du wiki-eu.model behar, vocab.txt-ekin nahikoa da

!python bert/create_pretraining_data.py \
  --input_file=wiki_00 \
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

#############################################################
#############################################################